#Part 2

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats
import numpy as np
from math import sqrt

In [55]:
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    
    return RMSE

In [2]:
#read in your London Commuting Data
cdata = pd.read_csv("london_flows.csv")

In [3]:
#look at the flows that originate from the city of London
cdata.head(33)

,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167
5,Abbey Road,Crossharbour,1,599,1208,6686.475560
6,Abbey Road,Custom House,0,599,845,3824.855630
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909
8,Abbey Road,Cyprus,7,599,850,6532.099618
9,Abbey Road,Devons Road,1,599,611,3958.324171


## I randomly choose 7 stations including Canary Wharf to run the models.

In [3]:
#we can select seven random boroughs
to_match_upd = ['Abbey Road', 'Canary Wharf', 'Waterloo', 
                'Westminster', 'Ruislip Manor', 
                'Euston', 'Ealing']

#subset the data by the 7 sample boroughs
#create cdatasub_new
#first the origins
cdatasub_new = cdata[cdata["station_origin"].isin(to_match_upd)]
#then the destinations
cdatasub_new = cdatasub_new[cdata["station_destination"].isin(to_match_upd)]


#remove intrap-borough flows
cdatasub_new = cdatasub_new[cdata["station_origin"] != cdata["station_destination"]]


beg = ["station_origin", "station_destination", "flows"] 
cols = beg + [col for col in cdatasub_new.columns.tolist() if col not in beg]

#re index the columns
cdatasub_new = cdatasub_new.reindex(columns = cols)


/tmp/ipykernel_242/3154729424.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cdatasub_new = cdatasub_new[cdata["station_destination"].isin(to_match_upd)]
/tmp/ipykernel_242/3154729424.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cdatasub_new = cdatasub_new[cdata["station_origin"] != cdata["station_destination"]]


In [58]:
#check the head of the data
cdatasub_new.head(10)

,station_origin,station_destination,flows,population,jobs,distance
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
8805,Canary Wharf,Abbey Road,2,14632,345,5086.514220
8893,Canary Wharf,Euston,58,14632,16800,10053.818310
9026,Canary Wharf,Ruislip Manor,0,14632,408,32151.123373
9083,Canary Wharf,Waterloo,733,14632,23466,7433.985750
9099,Canary Wharf,Westminster,429,14632,15466,8284.345776
17517,Euston,Canary Wharf,478,17796,58772,10053.818310
17719,Euston,Waterloo,652,17796,23466,3881.020921
17732,Euston,Westminster,217,17796,15466,3865.357133
44111,Ruislip Manor,Canary Wharf,36,1399,58772,32151.123373


## Scenario A: assume that Canary Wharf has a 50% decrease in jobs after Brexit.

In [86]:
cdatasub_new[cdatasub_new['station_origin']=='Canary Wharf']['jobs']=0.5* cdatasub_new[cdatasub_new['station_origin']=='Canary Wharf']['jobs']

/tmp/ipykernel_4436/3786652970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdatasub_new[cdatasub_new['station_origin']=='Canary Wharf']['jobs']=0.5* cdatasub_new[cdatasub_new['station_origin']=='Canary Wharf']['jobs']


In [87]:
cdatasub_new[cdatasub_new['station_destination']=='Canary Wharf']['jobs']=0.5*cdatasub_new[cdatasub_new['station_destination']=='Canary Wharf']['jobs']

/tmp/ipykernel_4436/2429241751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdatasub_new[cdatasub_new['station_destination']=='Canary Wharf']['jobs']=0.5*cdatasub_new[cdatasub_new['station_destination']=='Canary Wharf']['jobs']


In [88]:
#now we can create a pivot table to turn paired list into a matrix, and compute the margin as well
cdatasubmat_new = pd.pivot_table(cdatasub_new, values ="flows", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
cdatasubmat_new

/tmp/ipykernel_4436/514196163.py:2: FutureWarning: The provided callable <function sum at 0x7fb7fc1a98a0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  cdatasubmat_new = pd.pivot_table(cdatasub_new, values ="flows", index="station_origin", columns = "station_destination",
/tmp/ipykernel_4436/514196163.py:2: FutureWarning: The provided callable <function sum at 0x7fb7fc1a98a0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  cdatasubmat_new = pd.pivot_table(cdatasub_new, values ="flows", index="station_origin", columns = "station_destination",
/tmp/ipykernel_4436/514196163.py:2: FutureWarning: The provided callable <function sum at 0x7fb7fc1a98a0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used direc

station_destination,Abbey Road,Canary Wharf,Euston,Ruislip Manor,Waterloo,Westminster,All
station_origin,,,,,,,
Abbey Road,NaN,1.0,NaN,NaN,NaN,NaN,1
Canary Wharf,2.0,NaN,58.0,0.0,733.0,429.0,1222
Euston,NaN,478.0,NaN,NaN,652.0,217.0,1347
Ruislip Manor,NaN,36.0,NaN,NaN,16.0,17.0,69
Waterloo,NaN,8085.0,1190.0,2.0,NaN,717.0,9994
Westminster,NaN,257.0,7.0,0.0,15.0,NaN,279
All,2.0,8857.0,1255.0,2.0,1416.0,1380.0,12912


In [89]:

cdatasub_new['log_jobs'] = np.log(cdatasub_new['jobs'])
cdatasub_new['log_distance'] = np.log(cdatasub_new['distance'])



In [90]:
#create the formula (the "-1" indicates no intercept in the regression model).
formula = 'flows ~ population + log_jobs + log_distance-1'


In [91]:
#run a production constrained sim
prodSim = smf.glm(formula = formula, data=cdatasub_new, family=sm.families.Poisson()).fit()


In [92]:
#let's have a look at it's summary
print(prodSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                   20
Model:                            GLM   Df Residuals:                       17
Model Family:                 Poisson   Df Model:                            2
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -4052.7
Date:                Tue, 16 Apr 2024   Deviance:                       7993.7
Time:                        18:31:42   Pearson chi2:                 6.97e+03
No. Iterations:                     6   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
population    4.356e-05   3.45e-07    126.384   

In [93]:
#create some Oi and Dj columns in the dataframe and store row and column totals in them:
#to create O_i, take cdatasub ...then... group by origcodenew ...then... summarise by calculating the sum of Total
O_i = pd.DataFrame(cdatasub_new.groupby(["station_origin"])["flows"].agg(np.sum))
O_i.rename(columns={"flows":"O_i"}, inplace = True)
cdatasub_new = cdatasub_new.merge(O_i, on = "station_origin", how = "left" )

D_j = pd.DataFrame(cdatasub_new.groupby(["station_destination"])["flows"].agg(np.sum))
D_j.rename(columns={"flows":"D_j"}, inplace = True)
cdatasub_new = cdatasub_new.merge(D_j, on = "station_destination", how = "left" )

/tmp/ipykernel_4436/1102129585.py:3: FutureWarning: The provided callable <function sum at 0x7fb7fc1a98a0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  O_i = pd.DataFrame(cdatasub_new.groupby(["station_origin"])["flows"].agg(np.sum))
/tmp/ipykernel_4436/1102129585.py:7: FutureWarning: The provided callable <function sum at 0x7fb7fc1a98a0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  D_j = pd.DataFrame(cdatasub_new.groupby(["station_destination"])["flows"].agg(np.sum))


In [94]:
cdatasub_new.head()

,station_origin,station_destination,flows,population,jobs,distance,log_jobs,log_distance,O_i,D_j
0,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981421,8.534348,1,8857
1,Canary Wharf,Abbey Road,2,14632,345,5086.514220,5.843544,8.534348,1222,2
2,Canary Wharf,Euston,58,14632,16800,10053.818310,9.729134,9.215708,1222,1255
3,Canary Wharf,Ruislip Manor,0,14632,408,32151.123373,6.011267,10.378203,1222,2
4,Canary Wharf,Waterloo,733,14632,23466,7433.985750,10.063308,8.913817,1222,1416


In [96]:
alpha_i = prodSim.params[0:1]
gamma = prodSim.params[1]
beta = -prodSim.params[2]

/tmp/ipykernel_4436/1405065752.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gamma = prodSim.params[1]
/tmp/ipykernel_4436/1405065752.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = -prodSim.params[2]


In [97]:
beta

0.08912115315428104

###Senario B

In [8]:
import numpy as np

# Original cost function parameter
b_orig = 1.0

# Two new values for the increased transport cost scenario
b_high = 1.5  # Moderate increase
b_extreme = 2.0  # Extreme increase

# Function to compute gravity model flows
def gravity_model(origins, destinations, population, employment, distance, b):
    flows = np.zeros((len(origins), len(destinations)))
    for i, origin in enumerate(origins):
        for j, destination in enumerate(destinations):
            numerator = population[origin] * employment[destination]
            denominator = np.power(distance[i, j], b)
            flows[i, j] = numerator / denominator
    return flows

# Compute original flows take canary wharf to euston as an example
flows_orig = gravity_model("Canary Wharf", "Euston", 14632, 16800, 10053.818310, b_orig)

# Compute flows with increased transport cost
flows_high = gravity_model(origins, destinations, population, employment, distance, b_high)
flows_extreme = gravity_model(origins, destinations, population, employment, distance, b_extreme)

# Print the original and new flow distributions
print("Original flow distribution:")
print(flows_orig)

print("\nFlow distribution with moderate transport cost increase (b = 1.5):")
print(flows_high)

print("\nFlow distribution with extreme transport cost increase (b = 2.0):")
print(flows_extreme)

TypeError: 'int' object is not subscriptable